In [31]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" # see issue #152
os.environ["CUDA_VISIBLE_DEVICE"] = "-1" # for using only cpu
import tensorflow as tf
import numpy as np


In [35]:
a = tf.constant([2], dtype=tf.float32)
b = tf.constant([3], dtype=tf.float32)
c = tf.constant([4], dtype=tf.float32)

op = a*b+c
# op = tf.add(tf.multiply(a,b),c)

sess = tf.Session()
result = sess.run(op)
print(result)
sess.close()

[10.]


In [40]:
# Constant 1-D Tensor populated with value list.
tensor1 = tf.constant([1,2,3,4,5,6,7])

# Constant 2-D
tensor2 = tf.constant(-1.0, shape=[2,3]) # [[-1. -1. -1.], [-1. -1. -1.]]

sess = tf.Session()
print(sess.run(tensor1))
print(sess.run(tensor2))
print(tensor1)
print(tensor2)
sess.close()

[1 2 3 4 5 6 7]
[[-1. -1. -1.]
 [-1. -1. -1.]]
Tensor("Const_24:0", shape=(7,), dtype=int32)
Tensor("Const_25:0", shape=(2, 3), dtype=float32)


In [49]:
input_data = np.array([[1,1],[1,1]])
input_data2 = np.array([[1,1,1],[1,1,1]])

x = tf.placeholder(dtype = tf.float32)
k = tf.placeholder(dtype = tf.float32)
y = tf.matmul(x,k)

sess = tf.Session()
result = sess.run(y, feed_dict = {x:input_data, k:input_data2})

print(result)

[[2. 2. 2.]
 [2. 2. 2.]]


In [54]:
input_data = [1,2,3,4,5]

x = tf.placeholder(dtype=tf.float32)
W = tf.Variable([10], dtype=tf.float32)
W = W.assign_add([10.0])
y = W*x

# initialize all variables or error occurs
init = tf.global_variables_initializer()
sess.run(init)

result = sess.run(y, feed_dict = {x:input_data})

print(result)

[ 20.  40.  60.  80. 100.]


In [57]:
import scipy.misc

# download mnist data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST/', one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting MNIST/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting MNIST/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [65]:
mnist.train.images.shape

(55000, 784)

In [69]:
scipy.misc.toimage(mnist.train.images[999].reshape((28,28)))

c:\users\tony\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  """Entry point for launching an IPython kernel.


In [71]:
mnist.train.labels[999]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])

In [72]:
# dataset n is unknown
images = tf.placeholder(shape = [None, 784], dtype=tf.float32)
labels = tf.placeholder(shape = [None, 10], dtype = tf.float32)

# Weight initialization by normal distiribution with 
W = tf.Variable(tf.truncated_normal([784, 10], stddev=0.1))
b = tf.Variable(tf.zeros([10]))

In [75]:
logit = tf.nn.softmax(tf.matmul(images, W)+ b)
loss = -tf.reduce_sum(labels*tf.log(logit))

In [76]:
# Set optimize with loss function
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

In [77]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [78]:
# training over 1000 times
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_op, feed_dict = {images: batch_xs, labels: batch_ys})

In [79]:
correct_prediction = tf.equal(tf.argmax(logit, axis =1), tf.argmax(labels, axis=1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [84]:
print(sess.run(acc, feed_dict = {images:mnist.train.images, labels:mnist.train.labels}))

0.9159818


In [85]:
print(sess.run(acc, feed_dict = {images:mnist.test.images, labels:mnist.test.labels}))

0.9139


In [ ]:
# Practice MLP

In [182]:
# dataset n is unknown
images = tf.placeholder(shape = [None, 784], dtype=tf.float32)
labels = tf.placeholder(shape = [None, 10], dtype = tf.float32)

# Weight initialization by normal distiribution with 
W1 = tf.Variable(tf.truncated_normal([784, 150], stddev=0.1))
b1 = tf.Variable(tf.zeros([150]))
W2 = tf.Variable(tf.truncated_normal([150, 100], stddev=0.1))
b2 = tf.Variable(tf.zeros([100]))
W3 = tf.Variable(tf.truncated_normal([100, 10], stddev=0.1))
b3 = tf.Variable(tf.zeros([10]))

In [183]:
out1 = tf.nn.relu(tf.matmul(images, W1)+b1)
out2 = tf.nn.relu(tf.matmul(out1, W2)+b2)
logit = tf.matmul(out2, W3)+b3

loss = tf.reduce_sum(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)

In [184]:
train_op = tf.train.RMSPropOptimizer(0.003).minimize(loss)

In [185]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [186]:
for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(200, shuffle=True)
    _, batch_loss = sess.run([train_op, loss], feed_dict={images: batch_xs, labels: batch_ys})
print(batch_loss)

3.6415105


In [187]:
acc = tf.reduce_mean(tf.cast(
    tf.equal(tf.argmax(logit, axis=1), tf.argmax(labels, axis=1)),
    tf.float32))

In [188]:
print(sess.run(acc, feed_dict={images: mnist.train.images, labels: mnist.train.labels}))

0.9944182


In [189]:
print(sess.run(acc, feed_dict={images: mnist.test.images, labels: mnist.test.labels}))

0.9776


In [96]:
################### MNIST CNN

In [231]:
def weight_variable(shape, name='weights'):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial, name=name)

def bias_variable(shape, name='biases'):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=name)

In [232]:
def conv(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [233]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [234]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [235]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])
fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
drop_out = tf.nn.dropout(fc_layer1, keep_prob)

logit = tf.matmul(drop_out, W_fc2) + b_fc2

In [236]:

loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)
train_op = tf.train.AdamOptimizer(3e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

BATCH_SIZE = 100
max_epoch = 300
total_batch = int(mnist.train.num_examples/BATCH_SIZE)

# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for i in range(max_epoch):
    batch = mnist.train.next_batch(BATCH_SIZE, shuffle=True)
    sess.run(train_op, 
         feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
    )
    train_acc = sess.run(accuracy,
        feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
    )
    print('step %d, training accuracy %g' % (i, train_acc))
    if i % 100 == 0:
        test_acc = sess.run(accuracy, 
             feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
        )
        print('test accuracy %g' % test_acc)
test_acc = sess.run(accuracy, 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)

step 0, training accuracy 0.1
test accuracy 0.0765
step 1, training accuracy 0.09
step 2, training accuracy 0.05
step 3, training accuracy 0.17
step 4, training accuracy 0.13
step 5, training accuracy 0.1
step 6, training accuracy 0.21
step 7, training accuracy 0.3
step 8, training accuracy 0.27
step 9, training accuracy 0.32
step 10, training accuracy 0.21
step 11, training accuracy 0.19
step 12, training accuracy 0.27
step 13, training accuracy 0.39
step 14, training accuracy 0.39
step 15, training accuracy 0.36
step 16, training accuracy 0.42
step 17, training accuracy 0.5
step 18, training accuracy 0.54
step 19, training accuracy 0.48
step 20, training accuracy 0.44
step 21, training accuracy 0.61
step 22, training accuracy 0.53
step 23, training accuracy 0.57
step 24, training accuracy 0.64
step 25, training accuracy 0.6
step 26, training accuracy 0.56
step 27, training accuracy 0.65
step 28, training accuracy 0.57
step 29, training accuracy 0.57
step 30, training accuracy 0.67
st

step 251, training accuracy 0.96
step 252, training accuracy 0.97
step 253, training accuracy 0.94
step 254, training accuracy 0.95
step 255, training accuracy 0.96
step 256, training accuracy 0.88
step 257, training accuracy 0.94
step 258, training accuracy 0.95
step 259, training accuracy 0.92
step 260, training accuracy 0.94
step 261, training accuracy 0.94
step 262, training accuracy 0.94
step 263, training accuracy 0.92
step 264, training accuracy 0.9
step 265, training accuracy 0.91
step 266, training accuracy 0.95
step 267, training accuracy 0.95
step 268, training accuracy 0.96
step 269, training accuracy 0.91
step 270, training accuracy 0.9
step 271, training accuracy 0.94
step 272, training accuracy 0.92
step 273, training accuracy 0.93
step 274, training accuracy 0.95
step 275, training accuracy 0.94
step 276, training accuracy 0.97
step 277, training accuracy 0.95
step 278, training accuracy 0.91
step 279, training accuracy 0.91
step 280, training accuracy 0.95
step 281, tr

In [ ]:
############## Prediction with CNN

In [207]:
input_data = mnist.train.images[999:1000]
print(np.argmax(mnist.train.labels[999]))
scipy.misc.toimage(input_data[0].reshape((28,28)))

1


c:\users\tony\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `toimage` is deprecated!
`toimage` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use Pillow's ``Image.fromarray`` directly instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [208]:
pred = tf.argmax(logit, 1)
result = sess.run(pred, feed_dict={images: input_data, keep_prob: 1.0})
print('Prediction Result: {}'.format(result[0]))

Prediction Result: 1


In [ ]:
################# Tensorboard with CNN

In [209]:
images = tf.placeholder(shape=[None, 784], dtype=tf.float32)
reshape_images = tf.reshape(images, [-1, 28, 28, 1])

labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)

In [210]:
W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

In [211]:
conv_layer1 = tf.nn.relu(conv(reshape_images, W_conv1) + b_conv1)
pool_layer1 = max_pool(conv_layer1)

conv_layer2 = tf.nn.relu(conv(pool_layer1, W_conv2) + b_conv2)
pool_layer2 = max_pool(conv_layer2)

pool2_flat = tf.reshape(pool_layer2, [-1, 7*7*64])

with tf.name_scope('fc_layer') as scope:
    fc_layer1 = tf.nn.relu(tf.matmul(pool2_flat, W_fc1) + b_fc1)

with tf.name_scope('fc_layer') as scope:
    keep_prob = tf.placeholder(tf.float32)
    drop_out = tf.nn.dropout(fc_layer1, keep_prob)
    logit = tf.matmul(drop_out, W_fc2) + b_fc2

In [212]:
loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logit)
)
train_op = tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction = tf.equal(tf.argmax(logit, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.image('images', reshape_images)
tf.summary.scalar('xentropy', loss)
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

max_epoch = 500
# with tf.Session() as sess:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
test_writer = tf.summary.FileWriter('./test', sess.graph)
for i in range(max_epoch):
    batch = mnist.train.next_batch(50, shuffle=True)
    sess.run(train_op, 
         feed_dict={images: batch[0], labels: batch[1], keep_prob: 0.5}
    )
    train_acc = sess.run(accuracy,
        feed_dict={images: batch[0], labels: batch[1], keep_prob: 1.0}
    )
    print('step %d, training accuracy %g' % (i, train_acc))
    if i % 100 == 0:
        test_acc, summary = sess.run([accuracy, merged], 
             feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
        )
        print('test accuracy %g' % test_acc)
        test_writer.add_summary(summary, i)
test_acc, summary = sess.run([accuracy, merged], 
     feed_dict={images: mnist.test.images, labels: mnist.test.labels, keep_prob: 1.0}
)
print('test accuracy %g' % test_acc)
test_writer.add_summary(summary, i)

step 0, training accuracy 0.22
test accuracy 0.1278
step 1, training accuracy 0.14
step 2, training accuracy 0.18
step 3, training accuracy 0.28
step 4, training accuracy 0.2
step 5, training accuracy 0.2
step 6, training accuracy 0.18
step 7, training accuracy 0.14
step 8, training accuracy 0.26
step 9, training accuracy 0.34
step 10, training accuracy 0.22
step 11, training accuracy 0.28
step 12, training accuracy 0.26
step 13, training accuracy 0.26
step 14, training accuracy 0.34
step 15, training accuracy 0.38
step 16, training accuracy 0.46
step 17, training accuracy 0.24
step 18, training accuracy 0.48
step 19, training accuracy 0.34
step 20, training accuracy 0.28
step 21, training accuracy 0.52
step 22, training accuracy 0.54
step 23, training accuracy 0.46
step 24, training accuracy 0.48
step 25, training accuracy 0.66
step 26, training accuracy 0.58
step 27, training accuracy 0.5
step 28, training accuracy 0.58
step 29, training accuracy 0.48
step 30, training accuracy 0.58


step 252, training accuracy 0.9
step 253, training accuracy 0.88
step 254, training accuracy 0.92
step 255, training accuracy 0.94
step 256, training accuracy 0.94
step 257, training accuracy 0.88
step 258, training accuracy 0.88
step 259, training accuracy 0.88
step 260, training accuracy 0.92
step 261, training accuracy 0.9
step 262, training accuracy 0.92
step 263, training accuracy 0.88
step 264, training accuracy 1
step 265, training accuracy 0.94
step 266, training accuracy 0.94
step 267, training accuracy 0.92
step 268, training accuracy 0.94
step 269, training accuracy 0.9
step 270, training accuracy 0.92
step 271, training accuracy 0.92
step 272, training accuracy 0.94
step 273, training accuracy 0.8
step 274, training accuracy 0.88
step 275, training accuracy 0.94
step 276, training accuracy 0.96
step 277, training accuracy 0.92
step 278, training accuracy 0.94
step 279, training accuracy 0.88
step 280, training accuracy 0.96
step 281, training accuracy 1
step 282, training a

In [213]:
summary

b'\n\x9a\x02\n\x0eimages/image/0"\x87\x02\x08\x1c\x10\x1c\x18\x01"\xfe\x01\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x00\x00\x00\xc5IDAT(\x91c`\x186\x80\x11B\x85\xa4>\xfb\xb1\xf4\xc5\x1d\xec\x92\xf7\x14\x18\x18\x18>_\x85\n>\xe9:\xc3\xc0\xc0\xc0\xc0\xc0\x02\xe1\xa6\xea_\xd32t\xb0x,\xcb\xc0\xf0\xe7\xb5$\xc3\xa33\xc8:\x19\x18\x18\x18\x18\x04\r\xcf\x9820\xfc\xb8u](g\x1a.g\x04\xff\xbd(\x84KN\xec\xe5\xff`(\x93\tC2[\xf4\xfdM\\\x1a\xad\x7f\xfe\xb3\xc3%\xc7\xd0\xfao7+.9\xce\xb3?\xacpj\xac\xfb\xb7\r\xa7\x9c\xf7\xef\x0f\x96\xb8\xe4\x84\xef\xfe[\x86K\x8e\xf9\xf4\xbf\xdb\xca\xb8$\xd5\xfe\xfd\xf3\xc5%\'\xff\xe0_1#.\xc9\xd6\x7f\xffLp\xc9\xd9~B\x97D\n[\x1b\x1e\x86\xbb_P$Y\x909\x17\x9d\xdf\xe12\x16\x13\x00\x00i\x11<\x90\x8e\t\xbc\xff\x00\x00\x00\x00IEND\xaeB`\x82\n\xc4\x02\n\x0eimages/image/1"\xb1\x02\x08\x1c\x10\x1c\x18\x01"\xa8\x02\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\x00\x00\x1c\x08\x00\x00\x00\x00Wf\x80H\x00\x00\x00\xefIDAT(\x91c